In [2]:
from reader import Reader
import matplotlib.pyplot as plt
import numpy as np
from math import sin, cos, sqrt, atan2, radians
from datetime import datetime
from datetime import date as date_creator
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from numba import njit
import pandas as pd
import os
from numba import jit
from tqdm import tqdm
from numba import njit
import igc_lib

%matplotlib inline

In [3]:
directory = 'C:/Users/barroisl/Documents/Stage_LEo/Vols/'
days = ['IGC_19-08-2023/','IGC_20-08-2023/','IGC_21-08-2023/','IGC_22-08-2023/','IGC_23-08-2023/','IGC_24-08-2023/']
type_vols = ['Parapente/','Planeur/']

interv_para = np.zeros((1,24*6-1))[0]
interv_plan = np.zeros((1,24*6-1))[0]
for i in tqdm(range(len(days))):
    for j in type_vols:
        liste_file = [f for f in os.listdir(directory+days[i]+j) if f.endswith('.igc')]
        for k in liste_file :
            flight = directory+days[i]+j+k
            cand = igc_lib.Flight.create_from_file(flight, config_class=igc_lib.FlightParsingConfig)
            if cand.valid == True :
                takeoff = cand.takeoff_fix.rawtime+86400*i
                landing = cand.landing_fix.rawtime + 86400*i
                takeoff_ind = int(takeoff//(60*60))
                landing_ind = int(landing//(60*60))
                if takeoff_ind != landing_ind :
                    for k in range(takeoff_ind,landing_ind):
                        if j == 'Parapente/':
                            interv_para[k] = interv_para[k] + 1
                        elif j == 'Planeur/':
                            interv_plan[k] = interv_plan[k] + 1
                else :
                        if j == 'Parapente/':
                            interv_para[takeoff_ind] = interv_para[takeoff_ind] + 1  
                        elif j == 'Planeur/':
                            interv_plan[takeoff_ind] = interv_plan[takeoff_ind] + 1      

for i in range(len(interv_plan)):
    if interv_plan[i] == 0 :
        interv_plan[i] = np.nan

    if interv_para[i] == 0 :
        interv_para[i] = np.nan

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]


NameError: name 'igc_lib' is not defined

In [ ]:
fig, ax1 = plt.subplots(figsize = (25,11))
size = 30
days = [19,20,21,22,23,24]
ticks_label = ['19' + '/08 ' + str(j*6) +'h' for j in range(4)]
for i in days[1:] :
    ticks_label_i = [str(i) + '/08 ' + str(j*6) +'h' for j in range(4)]
    ticks_label = ticks_label + ticks_label_i
ticks = np.arange(0,24*6,6)

color = 'tab:red'
ax1.set_ylabel('Amount of gliding flights', color=color, fontsize=size*1.2)
ax1.plot(interv_plan, color=color,linewidth=5)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_xticks(ticks, ticks_label, fontsize=size, rotation = 90)
ax1.set_yticks(np.arange(0,160,20), [str(i) for i in np.arange(0,160,20)], fontsize=size)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Amount of paragliding flights', color=color, fontsize=size*1.2)  # we already handled the x-label with ax1
ax2.plot(interv_para, color=color,linewidth=5)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_yticks(np.arange(0,600,100), [str(i) for i in np.arange(0,600,100)], fontsize=size)

fig.tight_layout()  # otherwise the right y-label is slightly clipped

plt.savefig(savefig_path+'disponibilité_journalière.png')